In [11]:
import torch as t
from torch import einsum
from einops import rearrange, reduce, repeat
import gpt_tests
from torch import nn
from torch.nn import Module
from math import sqrt

In [ ]:
class MultiHeadedAttention(Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size/num_heads
        self.linear_layer = nn.Linear(hidden_size, hidden_size*3)
        self.output_projection = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        product = self.linear_layer(x)
        good_format = rearrange(product, 'b n (qkv h p) -> qkv b h n p', qkv = 3, h = self.num_heads)
        queries, keys, values = good_format[0], good_format[1], good_format[2]
        attn_score = t.einsum('bhfp,bhtp -> bhft', keys, queries) / sqrt(self.head_size)
        attn_pattn = t.softmax(attn_score, dim=-2)
        # attn_pattn: b h n n; values: b h n p
        out_by_head = t.einsum('bhft,bhfp->bhtp', attn_pattn, values)
        out = rearrange(out_by_head, 'b h t p -> b t (hp)') # b n hidden_size
        
        return self.output_projection(out)
        
        
        

1.4142135623730951